# AutoML avec MLJAR

## Installation

In [ ]:
!pip install mljar-supervised

## Librairies

In [ ]:
import pandas as pd

In [ ]:
from supervised.automl import AutoML 

# Dataset

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-apr-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-apr-2021/test.csv")

In [ ]:
def titanic_features(dataframe) :
    df = dataframe
    df["Embarked"] = df["Embarked"].fillna("S")
    df["Sex"] = df["Sex"].map({"female":0, "male":1})
    df["Family"] = df["SibSp"]+df["Parch"]+1
    df['IsAlone'] = 0
    df['Surname'] = df.Name.str.split(',').str[0]
    df.loc[df['Family'] == 1, 'IsAlone'] = 1
    df["Deck"] = df["Cabin"].apply(lambda x : "0" if pd.isna(x) else x[0])
    df = df.drop(["PassengerId", "Name", "Ticket", "Cabin", "SibSp", "Parch"], axis=1)    
    return df

In [ ]:
train = titanic_features(train)
test = titanic_features(test)

In [ ]:
train.head(10)

In [ ]:
y_train = train["Survived"]
X_train = train.drop(["Survived"], axis=1)

## Exploration des données (EDA)

In [ ]:
from supervised.preprocessing.eda import EDA

In [ ]:
EDA.extensive_eda(X_train,y_train,save_path=".")

In [ ]:
automl = AutoML(mode="Compete", eval_metric="accuracy") 

In [ ]:
automl.fit(X_train, y_train)

In [ ]:
automl.report()

## Soumission

In [ ]:
predictions = automl.predict(test)

In [ ]:
sample_submission = pd.read_csv('../input/tabular-playground-series-apr-2021/sample_submission.csv')

In [ ]:
sample_submission['Survived'] = predictions
sample_submission.to_csv('submission.csv',index=False)